In [1]:
import os
import sys
IN_COLAB = os.getenv("COLAB_RELEASE_TAG")
if IN_COLAB:
    !git clone https://github.com/cottascience/crosstalk-q1-2025
    sys.path.append('./crosstalk-q1-2025')
else:
    sys.path.append('..')

In [14]:
# Download the file from google drive
import gdown
import os


file_ids = {'test' : '19oR_A2UNUvy7pyL3J11oixkmF1m-bZBl',
            'train':'11S5p0QgP1X9rOFiIjNSLydLenJwm7hle'}

for name, file_id in file_ids.items():
    filename = f'crosstalk_{name}.parquet'
    if not os.path.exists(filename):
        gdown.download(id=file_id, output=filename, quiet=False)

Downloading...
From (original): https://drive.google.com/uc?id=19oR_A2UNUvy7pyL3J11oixkmF1m-bZBl
From (redirected): https://drive.google.com/uc?id=19oR_A2UNUvy7pyL3J11oixkmF1m-bZBl&confirm=t&uuid=9431f9c5-731d-44b7-a4e8-f77d59d1b5fe
To: /home/beangoben/github/crosstalk-q1-2025/notebooks/crosstalk_test.parquet
100%|██████████████████████████████████████| 1.52G/1.52G [01:02<00:00, 24.4MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=11S5p0QgP1X9rOFiIjNSLydLenJwm7hle
From (redirected): https://drive.google.com/uc?id=11S5p0QgP1X9rOFiIjNSLydLenJwm7hle&confirm=t&uuid=25223ec8-a940-4f7a-84e7-f56df1e2b2f2
To: /home/beangoben/github/crosstalk-q1-2025/notebooks/crosstalk_train.parquet
100%|██████████████████████████████████████| 1.97G/1.97G [01:22<00:00, 24.0MB/s]


In [15]:
import dataset
print(dataset.FINGERPRINT_TYPES)

['ATOMPAIR', 'MACCS', 'ECFP6', 'ECFP4', 'FCFP4', 'FCFP6', 'TOPTOR', 'RDK', 'AVALON']


In [16]:
filename = 'crosstalk_train.parquet'

In [17]:
y = dataset.load_y(filename)
y.shape

(375595, 1)

In [18]:
x_cols = ['MACCS']
x = dataset.load_x(test_file, x_cols)
x.shape

Total rows: 339204
Expected Memory for inputs: 0.21 GBs


  0%|          | 0/339 [00:00<?, ?it/s]

(339204, 167)

In [ ]:
# Test

In [ ]:
filename = 'crosstalk_train.parquet'
